In [6]:
import open3d as o3d
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


In [7]:
# from sklearn.neighbors import NearestNeighbors

# # Определение числа ближайших соседей
# n_neighbors = 1

# # Поиск ближайших соседей для каждой точки
# nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
# distances, indices = nbrs.kneighbors(cut_scaled_points_np)

# # Вычисление среднего расстояния до n_neighbors точек для каждой точки
# avg_distances = np.mean(distances, axis=1)

# # Определение порогового значения для удаления редких точек (можно настроить)
# threshold_low = np.percentile(avg_distances, 2)

# # Отфильтровать точки по пороговому значению
# filtered_indices = np.where(avg_distances <= threshold_low)[0]
# filtered_cut_points_np = cut_points_np[filtered_indices]
# filtered_cut_scaled_points_np = cut_scaled_points_np[filtered_indices]

# # Произвести повторное масштабирование, если это необходимо
# # filtered_scaled_points_np = scaler.fit_transform(filtered_cut_points_np)

# # Применение DBSCAN на отфильтрованных точках
# eps = 0.2  # Пример значения для параметра epsilon
# min_samples = 20  # Пример значения для параметра min_samples
# dbscan = DBSCAN(eps=eps, min_samples=min_samples)
# cut_labels = dbscan.fit_predict(filtered_cut_scaled_points_np)

# # Создание вектора цветов для обрезанных точек
# cut_num_points = len(filtered_cut_points_np)
# cut_colors = np.zeros((cut_num_points, 3))
# for label in np.unique(cut_labels):
#     if label == -1:
#         continue
#     mask = cut_labels == label
#     cut_colors[mask] = np.random.rand(1, 3)

# # Создание облака точек после обрезки
# cut_pcd = o3d.geometry.PointCloud()
# cut_pcd.points = o3d.utility.Vector3dVector(filtered_cut_points_np)
# cut_pcd.colors = o3d.utility.Vector3dVector(cut_colors)

# # Визуализация кластеров после обрезки
# o3d.visualization.draw_geometries([cut_pcd])



In [8]:
# cutting_height = -1.2  # Значение для отсечения снизу

# # Отсечение точек снизу по оси Z
# cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
# cut_points_np = points_np[cut_indices]
# cut_scaled_points_np = scaled_points_np[cut_indices]
# cut_labels = labels[cut_indices]

# # Создание вектора цветов для обрезанных точек
# cut_num_points = len(cut_points_np)
# cut_colors = np.zeros((cut_num_points, 3))
# for label in np.unique(cut_labels):
#     if label == -1:
#         continue
#     mask = cut_labels == label
#     cut_colors[mask] = np.random.rand(1, 3)

# # Создание облака точек после обрезки
# cut_pcd = o3d.geometry.PointCloud()
# cut_pcd.points = o3d.utility.Vector3dVector(cut_points_np)
# cut_pcd.colors = o3d.utility.Vector3dVector(cut_colors)

# # Визуализация кластеров после обрезки
# o3d.visualization.draw_geometries([cut_pcd])


In [9]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import open3d as o3d
import numpy as np



clustered_ply_file = 'D:/Cluster_proj/VIPCClustering/data/KITTI/velodyne/000008.bin'
# Загрузка данных из файла
with open(clustered_ply_file, 'rb') as f:
    # Чтение данных из файла в массив numpy
    velo_points = np.fromfile(f, dtype=np.float32).reshape(-1, 4)
# Загрузка облака точек из файла .bin
velo_points = np.fromfile(clustered_ply_file, dtype=np.float32).reshape(-1, 4)
points_np = velo_points[:, :3]



# Загрузка облака точек
# pcd = o3d.io.read_point_cloud(clustered_ply_file)
# points_np = np.asarray(pcd.points)

# Масштабирование данных
scaler = StandardScaler()
scaled_points_np = scaler.fit_transform(points_np)

# Определение высоты отсечения
cutting_height = -1.2  # Значение для отсечения снизу

# Отсечение точек снизу по оси Z
cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
cut_points_np = points_np[cut_indices]
cut_scaled_points_np = scaled_points_np[cut_indices]

# Определение числа ближайших соседей
n_neighbors = 1

# Поиск ближайших соседей для каждой точки
nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cut_scaled_points_np)
distances, indices = nbrs.kneighbors(cut_scaled_points_np)

# Вычисление среднего расстояния до n_neighbors точек для каждой точки
avg_distances = np.mean(distances, axis=1)

# Определение порогового значения для удаления редких точек (можно настроить)
threshold_low = np.percentile(avg_distances, 1)

# Отфильтровать точки по пороговому значению
filtered_indices = np.where(avg_distances <= threshold_low)[0]
filtered_cut_points_np = cut_points_np[filtered_indices]
filtered_cut_scaled_points_np = cut_scaled_points_np[filtered_indices]

# Применение DBSCAN на отфильтрованных точках
eps = 0.1  # Пример значения для параметра epsilon
min_samples = 10  # Пример значения для параметра min_samples
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
cut_labels = dbscan.fit_predict(filtered_cut_scaled_points_np)

# Создание вектора цветов для обрезанных точек
cut_num_points = len(filtered_cut_points_np)
cut_colors = np.zeros((cut_num_points, 3))
for label in np.unique(cut_labels):
    if label == -1:
        continue
    mask = cut_labels == label
    cut_colors[mask] = np.random.rand(1, 3)

# Создание облака точек после обрезки
cut_pcd = o3d.geometry.PointCloud()
cut_pcd.points = o3d.utility.Vector3dVector(filtered_cut_points_np)
cut_pcd.colors = o3d.utility.Vector3dVector(cut_colors)

# Визуализация кластеров после обрезки
o3d.visualization.draw_geometries([cut_pcd])


In [10]:
import open3d as o3d
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.spatial.distance import pdist, squareform



clustered_ply_file = 'D:/Cluster_proj/VIPCClustering/data/KITTI/velodyne/000008.bin'
# Загрузка данных из файла
with open(clustered_ply_file, 'rb') as f:
    # Чтение данных из файла в массив numpy
    velo_points = np.fromfile(f, dtype=np.float32).reshape(-1, 4)
# Загрузка облака точек из файла .bin
velo_points = np.fromfile(clustered_ply_file, dtype=np.float32).reshape(-1, 4)
points_np = velo_points[:, :3]



# Загрузка облака точек
# clustered_ply_file = 'clustered_point_cloud.ply'
# pcd = o3d.io.read_point_cloud(clustered_ply_file)
# points_np = np.asarray(pcd.points)

# Масштабирование данных
scaler = StandardScaler()
scaled_points_np = scaler.fit_transform(points_np)

# Определение высоты отсечения
cutting_height = -1.2  # Значение для отсечения снизу

# Отсечение точек снизу по оси Z
cut_indices = np.where(points_np[:, 2] > cutting_height)[0]
cut_points_np = points_np[cut_indices]
cut_scaled_points_np = scaled_points_np[cut_indices]

# Кластеризация DBSCAN для обрезанных точек
eps = 0.2  # Пример значения для параметра epsilon
min_samples = 5 # Пример значения для параметра min_samples
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
cut_labels = dbscan.fit_predict(cut_scaled_points_np)

# Создание вектора цветов для обрезанных точек
cut_num_points = len(cut_points_np)
cut_colors = np.zeros((cut_num_points, 3))
for label in np.unique(cut_labels):
    if label == -1:
        continue
    mask = cut_labels == label
    cut_colors[mask] = np.random.rand(1, 3)

# Создание облака точек после обрезки
cut_pcd = o3d.geometry.PointCloud()
cut_pcd.points = o3d.utility.Vector3dVector(cut_points_np)
cut_pcd.colors = o3d.utility.Vector3dVector(cut_colors)

# Визуализация кластеров после обрезки
o3d.visualization.draw_geometries([cut_pcd])


In [11]:
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score

# Silhouette Score
silhouette_avg = silhouette_score(filtered_cut_scaled_points_np, cut_labels)
print("Silhouette Score:", silhouette_avg)

# Dunn Index (requires installation of `pyclustertend` package)
try:
    from pyclustertend import dunn
    dunn_index = dunn(filtered_cut_scaled_points_np, cut_labels)
    print("Dunn Index:", dunn_index)
except ImportError:
    print("Dunn Index calculation requires the 'pyclustertend' package, please install it to calculate Dunn Index.")

# Davies-Bouldin Index
davies_bouldin_index = davies_bouldin_score(filtered_cut_scaled_points_np, cut_labels)
print("Davies-Bouldin Index:", davies_bouldin_index)

# C-Index (requires installation of `cluster-validation` package)
try:
    from clusteval import c_index
    c_index_value = c_index(filtered_cut_scaled_points_np, cut_labels)
    print("C-Index:", c_index_value)
except ImportError:
    print("C-Index calculation requires the 'cluster-validation' package, please install it to calculate C-Index.")

# Calinski–Harabasz Index
calinski_harabasz_index = calinski_harabasz_score(filtered_cut_scaled_points_np, cut_labels)
print("Calinski–Harabasz Index:", calinski_harabasz_index)


Silhouette Score: -0.09813004
Dunn Index calculation requires the 'pyclustertend' package, please install it to calculate Dunn Index.
Davies-Bouldin Index: 1.3492844040819187
C-Index calculation requires the 'cluster-validation' package, please install it to calculate C-Index.
Calinski–Harabasz Index: 968.9286820565736


In [12]:
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score

# Silhouette Score
silhouette_avg = silhouette_score(filtered_cut_scaled_points_np, cut_labels)
print("Silhouette Score:", silhouette_avg)

# Silhouette:
# The silhouette score measures how similar an object is to its own cluster compared to other clusters. 
# It ranges from -1 to 1, where a high value indicates that the object is well matched to its own cluster 
# and poorly matched to neighboring clusters.

# Dunn Index (requires installation of `pyclustertend` package)
try:
    from pyclustertend import dunn
    dunn_index = dunn(filtered_cut_scaled_points_np, cut_labels)
    print("Dunn Index:", dunn_index)
except ImportError:
    print("Dunn Index calculation requires the 'pyclustertend' package, please install it to calculate Dunn Index.")

# Dunn Index:
# The Dunn index measures the compactness and separation of clusters. 
# It is defined as the ratio of the smallest distance between points not in the same cluster to the 
# largest intra-cluster distance.

# Davies-Bouldin Index
davies_bouldin_index = davies_bouldin_score(filtered_cut_scaled_points_np, cut_labels)
print("Davies-Bouldin Index:", davies_bouldin_index)

# Davies-Bouldin Index:
# The Davies-Bouldin index measures the average similarity between each cluster and its most similar cluster. 
# Lower values of the Davies-Bouldin index indicate better clustering.

# C-Index (requires installation of `cluster-validation` package)
try:
    from clusteval import c_index
    c_index_value = c_index(filtered_cut_scaled_points_np, cut_labels)
    print("C-Index:", c_index_value)
except ImportError:
    print("C-Index calculation requires the 'cluster-validation' package, please install it to calculate C-Index.")

# Calinski–Harabasz Index
calinski_harabasz_index = calinski_harabasz_score(filtered_cut_scaled_points_np, cut_labels)
print("Calinski–Harabasz Index:", calinski_harabasz_index)

# Calinski–Harabasz Index:
# The Calinski–Harabasz index measures the ratio of between-cluster dispersion to within-cluster dispersion. 
# Higher values indicate better clustering.


Silhouette Score: -0.09813004
Dunn Index calculation requires the 'pyclustertend' package, please install it to calculate Dunn Index.
Davies-Bouldin Index: 1.3492844040819187
C-Index calculation requires the 'cluster-validation' package, please install it to calculate C-Index.
Calinski–Harabasz Index: 968.9286820565736


In [16]:
import numpy as np

def cevi_index(X, labels):
    clusters = np.unique(labels)
    cevi_values = []
    for cluster in clusters:
        cluster_points = X[labels == cluster]
        if len(cluster_points) <= 1:
            cevi_values.append(0)  # Not enough data for diversity estimation
        else:
            cluster_variance = np.var(cluster_points, axis=0)
            cevi_values.append(np.sum(cluster_variance))

    return np.mean(cevi_values)

# Compute CEVI Index

cevi_value = cevi_index(filtered_cut_points_np, cut_labels)
print("CEVI Index:", cevi_value)


CEVI Index: 14.145626


In [14]:
import numpy as np

def variety_index(X, labels):
    clusters = np.unique(labels)
    diversity_per_cluster = []
    for cluster in clusters:
        cluster_points = X[labels == cluster]
        if len(cluster_points) <= 1:
            diversity_per_cluster.append(0)  # Not enough data to assess diversity
        else:
            distances = np.linalg.norm(cluster_points[:, None] - cluster_points, axis=-1)
            diversity_per_cluster.append(np.std(distances))

    return np.mean(diversity_per_cluster)

# Calculation of "Variety Index"
variety_index_value = variety_index(filtered_cut_points_np, cut_labels)
print("Variety Index:", variety_index_value)


Variety Index: 1.1655959
